###    Mixins

A Mixin is a class that provides additional functionality to another class through inheritance, but it is not intended to be instantiated on its own. In Selenium Python, Mixins can be a powerful way to organize reusable components for common interactions, such as working with elements, form inputs, or custom utilities.

Mixins are small, reusable classes focused on adding specific functionality, whereas multiple inheritance combines broad functionalities from multiple parent classes.  
Mixins are easier to use, avoid complexity, and promote modularity.  
Choose Mixins for focused behaviors and multiple inheritance for combining full classes with shared functionality.  

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# Define Mixins
class ElementInteractionMixin:
    def click_element(self, by_locator):
        self.driver.find_element(*by_locator).click()

    def enter_text(self, by_locator, text):
        self.driver.find_element(*by_locator).send_keys(text)

    def get_element_text(self, by_locator):
        return self.driver.find_element(*by_locator).text


class WaitMixin:
    def wait_for_element_visible(self, by_locator, timeout=10):
        WebDriverWait(self.driver, timeout).until(
            EC.visibility_of_element_located(by_locator)
        )

    def wait_and_click(self, by_locator, timeout=10):
        WebDriverWait(self.driver, timeout).until(
            EC.element_to_be_clickable(by_locator)
        ).click()

# Base Page
class BasePage(ElementInteractionMixin, WaitMixin):
    def __init__(self, driver):
        self.driver = driver

# Page Objects
class LoginPage(BasePage):
    username_field = (By.ID, "username")
    password_field = (By.ID, "password")
    login_button = (By.ID, "loginButton")

    def login(self, username, password):
        self.enter_text(self.username_field, username)
        self.enter_text(self.password_field, password)
        self.click_element(self.login_button)


class DashboardPage(BasePage):
    logout_button = (By.ID, "logoutButton")

    def logout(self):
        self.wait_and_click(self.logout_button)


# Test Script
if __name__ == "__main__":
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach",True)
    #options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(options=options)
    driver.get("https://example.com/login")

    # Login
    login_page = LoginPage(driver)
    login_page.login("test_user", "test_password")

    # Logout
    dashboard_page = DashboardPage(driver)
    dashboard_page.logout()

    # Close the browser
    driver.quit()


In [1]:
# Mixins for reusable functionality
class LoggingMixin:
    def log(self, action, message):
        print(f"[{action.upper()}]: {message}")

class ValidationMixin:
    def validate_email(self, email):
        # Simple email validation logic
        return "@" in email and "." in email

# Base classes for user roles
class User:
    def __init__(self, name, email):
        self.name = name
        self.email = email

    def display_user_info(self):
        print(f"Name: {self.name}, Email: {self.email}")

class Admin(User):
    def manage_users(self):
        print(f"Admin {self.name} is managing users.")

class Customer(User):
    def make_purchase(self):
        print(f"Customer {self.name} is making a purchase.")

# Combining Mixins with multiple inheritance
class AdminWithUtilities(Admin, LoggingMixin, ValidationMixin):
    def perform_admin_task(self, task):
        if not self.validate_email(self.email):
            self.log("validation failed", f"Invalid email for admin {self.name}.")
            return
        self.log("task started", f"Admin {self.name} started the task: {task}.")
        self.manage_users()
        self.log("task completed", f"Task '{task}' completed successfully.")

class CustomerWithUtilities(Customer, LoggingMixin, ValidationMixin):
    def perform_customer_task(self, task):
        if not self.validate_email(self.email):
            self.log("validation failed", f"Invalid email for customer {self.name}.")
            return
        self.log("task started", f"Customer {self.name} started the task: {task}.")
        self.make_purchase()
        self.log("task completed", f"Task '{task}' completed successfully.")

# Usage example
if __name__ == "__main__":
    # Admin scenario
    admin = AdminWithUtilities("Alice", "alice@company.com")
    admin.display_user_info()
    admin.perform_admin_task("Update user permissions")

    print()

    # Customer scenario
    customer = CustomerWithUtilities("Bob", "bob@example.com")
    customer.display_user_info()
    customer.perform_customer_task("Purchase a subscription")

    print()

    # Invalid email scenario
    faulty_admin = AdminWithUtilities("Charlie", "invalid_email")
    faulty_admin.perform_admin_task("Reset passwords")


Name: Alice, Email: alice@company.com
[TASK STARTED]: Admin Alice started the task: Update user permissions.
Admin Alice is managing users.
[TASK COMPLETED]: Task 'Update user permissions' completed successfully.

Name: Bob, Email: bob@example.com
[TASK STARTED]: Customer Bob started the task: Purchase a subscription.
Customer Bob is making a purchase.
[TASK COMPLETED]: Task 'Purchase a subscription' completed successfully.

[VALIDATION FAILED]: Invalid email for admin Charlie.
